In [1]:
# Inizializzazione (CPU 2:00 min, GPU 1:50 min)
!git clone https://github.com/abewley/sort.git
!pip install lap filterpy
! pip install matplotlib==3.2.1

Cloning into 'sort'...
remote: Enumerating objects: 208, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 208 (delta 2), reused 2 (delta 1), pack-reused 202 (from 1)
Receiving objects: 100% (208/208), 1.21 MiB | 18.71 MiB/s, done.
Resolving deltas: 100% (74/74), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lap: filename=lap-0.4.0-cp310-cp310-linux_x86_64.whl size=1628945 sha256=08a5b552e92b2f8948fdd1d88751258965fbb8c287b935baec5d420d687661f8
  Stored in directory: /root/.cache/pip/wheels/00/42/2e/9dfe19270eea279d79e84767ff0d7b8082c3bf776cad00e83d
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110459 sha256=52d78f550be053355c20d442ed9214f669abab46cda12189527d14566db74ad3
  Sto

In [1]:
# Installa YOLO5
!git clone https://github.com/ultralytics/yolov5
!pip install -r yolov5/requirements.txt


from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.90 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.0/112.6 GB disk)


In [2]:
# Importa da Drive il necessario
from google.colab import drive
import os
drive.mount('/content/drive')

#dataset_folder = '/content/drive/MyDrive/CV project/dataset/'
dataset_folder = '/content/drive/MyDrive/Simone/CV project/dataset/'

#os.chdir(dataset_folder)

Mounted at /content/drive


In [3]:
import matplotlib

In [5]:
import cv2
import numpy as np
import csv
import json
import os
import sys

import torch

from tqdm.notebook import tqdm

matplotlib.use('TKAgg')
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath('sort'))
os.chdir('/content/sort')


#vehicle_classes = {2: "Car", 3: "Motorcycle", 5: "Bus", 7: "Truck"}
vehicle_classes = {
    'articulated_truck': 0,
    'bus': 2,
    'car': 3,
    'motorcycle': 4,
    'motorized_vehicle': 5,
    'pickup_truck': 8,
    'single_unit_truck': 9,
    'work_van': 10
}

In [6]:
from sort import Sort

In [11]:
os.chdir('/content')

In [7]:
# Per evitare fastidiosi output durante l'utilizzo di YOLO
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

def export_results(folder_path, name, d):
    with open(folder_path + name + ".json", 'w') as f:
       f.write(json.dumps(d))

In [19]:
# YOLO
def load_yolo_model(weights_path):
    #model = torch.hub.load("ultralytics/yolov5", "yolov5m")
    model = torch.hub.load('yolov5', 'custom', path=weights_path, source='local', verbose=False)
    model.eval()
    return model

def detect_objects_in_frame(model, frame, classes):
    results = model(frame)

    detections = results.xyxy[0].cpu().numpy()

    class_ids = []
    boxes = []
    confidences = []

    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        if conf > 0.5 and cls in list(classes.values()):
            boxes.append([x1, y1, x2, y2])
            confidences.append(conf.item())
            class_ids.append(int(cls))

    return boxes, class_ids, confidences

In [9]:
# Da ogni frame ottieni dati dei veicoli
def get_vehicles_position(model, tracker, frame, mask):

    # Applica YOLO
    roi_frame = cv2.bitwise_and(frame, frame, mask=mask)
    boxes, classes, confs = detect_objects_in_frame(model, roi_frame, vehicle_classes)
    if len(boxes) != 0:
        tracked_objects = tracker.update(np.array(boxes))
    else:
        return {}

    objects = {}
    for obj, cls, conf in zip(tracked_objects, classes, confs):
        x1, y1, x2, y2, obj_id = obj
        objects[obj_id] = {"id": obj_id, "class": cls, "bbox": [x1, y1, x2, y2], "conf": conf}

    return objects

In [20]:
# MAIN
folder_path = dataset_folder + "publishing_data/Videos/"
#folder_path = "/content/drive/MyDrive/publishing_data/Videos/"
frame_rate = 120
mask_path = dataset_folder + "_ROI.png"
weights_path = dataset_folder + "best.pt"
output_folder = dataset_folder + "yo5_mio_10_longest_results/"

model = load_yolo_model(weights_path)
tracker = Sort()

# Importa maschera
roi_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
mask = cv2.threshold(roi_mask, 127, 255, cv2.THRESH_BINARY)[1]


# Lista video elaborati e da elaborare
#videos_list = sorted([f for f in os.listdir(folder_path) if f.lower().endswith('.mov')])
videos_list = ['IMG_0507.MOV', 'IMG_0518.MOV', 'IMG_0527.MOV', 'IMG_0530.MOV', 'IMG_0539.MOV', 'IMG_0601.MOV', 'IMG_0611.MOV', 'IMG_0629.MOV', 'IMG_0636.MOV', 'IMG_0642.MOV']

# Inizializza csv di output
exporting_data = []

fatti = 0
for video_file in videos_list:
    if fatti < 0:
        fatti += 1
        continue
    video_path = os.path.join(folder_path, video_file)
    print(f"Elaborazione {video_path}")
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_number = 1

    history = {}
    while cap.isOpened():
        if frame_number % 120 == 0:
            print(f"{frame_number}/{total_frames}\t{frame_number // 120} s")
        ret, frame = cap.read()
        if not ret:
            break

        history[frame_number] = get_vehicles_position(model, tracker, cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), mask)

        frame_number += 1

    cap.release()
    #break   # fa solo il primo video

    export_results(output_folder, os.path.splitext(video_file)[0], history)

YOLOv5 🚀 v7.0-365-g12b577c8 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 212 layers, 20893344 parameters, 0 gradients, 48.0 GFLOPs
Adding AutoShape... 


Elaborazione /content/drive/MyDrive/Simone/CV project/dataset/publishing_data/Videos/IMG_0507.MOV
120/10120	1 s
240/10120	2 s
360/10120	3 s
480/10120	4 s
600/10120	5 s
720/10120	6 s
840/10120	7 s
960/10120	8 s
1080/10120	9 s
1200/10120	10 s
1320/10120	11 s
1440/10120	12 s
1560/10120	13 s
1680/10120	14 s
1800/10120	15 s
1920/10120	16 s
2040/10120	17 s
2160/10120	18 s
2280/10120	19 s
2400/10120	20 s
2520/10120	21 s
2640/10120	22 s
2760/10120	23 s
2880/10120	24 s
3000/10120	25 s
3120/10120	26 s
3240/10120	27 s
3360/10120	28 s
3480/10120	29 s
3600/10120	30 s
3720/10120	31 s
3840/10120	32 s
3960/10120	33 s
4080/10120	34 s
4200/10120	35 s
4320/10120	36 s
4440/10120	37 s
4560/10120	38 s
4680/10120	39 s
4800/10120	40 s
4920/10120	41 s
5040/10120	42 s
5160/10120	43 s
5280/10120	44 s
5400/10120	45 s
5520/10120	46 s
5640/10120	47 s
5760/10120	48 s
5880/10120	49 s
6000/10120	50 s
6120/10120	51 s
6240/10120	52 s
6360/10120	53 s
6480/10120	54 s
6600/10120	55 s
6720/10120	56 s
6840/10120	57 s
6960/10